# Docteur vs ChatGPT: Chatbot médical

In [15]:
%pip install semantic-kernel openai python-dotenv --quiet


Note: you may need to restart the kernel to use updated packages.


## Import des bibliothèques


In [16]:
import os
import logging
import asyncio
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from typing import Annotated

In [17]:
# Charger les variables d'environnement
load_dotenv()

True

## Configuration des logs

In [18]:
# Configuration des logs
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('MedicalAI')

## Création du kernel Semantic Kernel

In [19]:
# Création du kernel Semantic Kernel
def create_kernel():
    kernel = Kernel()
    kernel.add_service(OpenAIChatCompletion(
        service_id="openai",
        ai_model_id="gpt-4o-mini",  # Modifier si besoin
        api_key=os.getenv("OPENAI_API_KEY")
    ))
    return kernel

## Définition des plugins à chaque agent

In [20]:
class DoctorPlugin:
    """Plugin permettant au médecin de poser des questions complémentaires sur les symptômes."""
    @kernel_function(description="Pose des questions supplémentaires pour affiner le diagnostic.")
    def ask_followup_questions(self, symptom: Annotated[str, "Symptôme décrit par l'utilisateur"]) -> str:
        """Retourne une question en fonction du symptôme mentionné."""
        questions_map = {
            "fièvre": "Depuis combien de temps avez-vous de la fièvre ?",
            "maux de tête": "Avez-vous une sensibilité à la lumière ou au bruit ?",
            "douleur thoracique": "La douleur est-elle aiguë ou diffuse ?",
        }
        return questions_map.get(symptom.lower(), "Pouvez-vous donner plus de détails sur vos symptômes ?")

class MedicalAIPlugin:
    """Plugin qui analyse la gravité des symptômes."""
    @kernel_function(description="Vérifie la gravité d'un symptôme médical.")
    def check_symptom_severity(self, symptom: Annotated[str, "Symptôme décrit par l'utilisateur"]) -> str:
        """Retourne une évaluation de la gravité du symptôme."""
        severity_map = {
            "fièvre": "Modérée",
            "maux de tête": "Léger",
            "douleur thoracique": "Sévère",
            "perte de connaissance": "Critique"
        }
        return severity_map.get(symptom.lower(), "Inconnu - consultez un médecin.")

class PharmacistPlugin:
    """Plugin qui recommande des médicaments en fonction du diagnostic."""
    @kernel_function(description="Recommande un médicament adapté à un symptôme.")
    def recommend_medication(self, symptom: Annotated[str, "Symptôme décrit par l'utilisateur"]) -> str:
        """Retourne une suggestion de médicament (avec précautions)."""
        medication_map = {
            "fièvre": "Paracétamol (500mg, toutes les 6h, max 3 jours)",
            "maux de tête": "Ibuprofène (200mg, toutes les 8h, avec précaution si problème gastrique)",
            "douleur thoracique": "Aucun médicament recommandé - Consultez un médecin immédiatement",
        }
        return medication_map.get(symptom.lower(), "Aucun médicament recommandé - Consultez un pharmacien.")


## Création du Kernel

In [21]:
# Création du kernel
kernel = create_kernel()


# Ajout des plugins pour chaque agent

In [22]:
# Ajout des plugins pour chaque agent
kernel.add_plugin(DoctorPlugin(), plugin_name="doctor")
kernel.add_plugin(MedicalAIPlugin(), plugin_name="medical")
kernel.add_plugin(PharmacistPlugin(), plugin_name="pharmacist")


KernelPlugin(name='pharmacist', description=None, functions={'recommend_medication': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='recommend_medication', plugin_name='pharmacist', description='Recommande un médicament adapté à un symptôme.', parameters=[KernelParameterMetadata(name='symptom', description="Symptôme décrit par l'utilisateur", default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': "Symptôme décrit par l'utilisateur"}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x116ecd460>, streaming_duration_histogram=<opentelemetry.

## Définition des prompts 

In [23]:
DOCTOR_PROMPT = """
Vous êtes un médecin généraliste. Vous posez d'abord des questions pour mieux comprendre les symptômes de l'utilisateur,
puis vous donnez un diagnostic probable basé sur votre expertise médicale. 
Ne donnez jamais de diagnostic sans avoir recueilli assez d'informations.
"""

AI_MEDICAL_PROMPT = """
Vous êtes une IA médicale spécialisée en diagnostic. Analysez les symptômes fournis et proposez un diagnostic basé sur des statistiques et des études médicales. 
Soyez clair et donnez plusieurs hypothèses si nécessaire.
"""

PHARMACIST_PROMPT = """
Vous êtes un pharmacien qualifié. En fonction du diagnostic fourni, vous recommandez les médicaments appropriés. 
Mentionnez toujours les précautions d'utilisation et la nécessité d'une consultation médicale avant la prise de médicaments.
"""


## Création des agents

In [24]:
# Création des agents
doctor_agent = ChatCompletionAgent(
    kernel=kernel,
    service_id="openai",
    name="Docteur_Humain",
    instructions=DOCTOR_PROMPT,
)

ai_medical_agent = ChatCompletionAgent(
    kernel=kernel,
    service_id="openai",
    name="IA_Medicale",
    instructions=AI_MEDICAL_PROMPT,
)

## Configuration pour que l'agent médical appelle automatiquement les plugins


In [25]:
settings = kernel.get_prompt_execution_settings_from_service_id("openai")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
ai_medical_agent.arguments = KernelArguments(settings=settings)

pharmacist_agent = ChatCompletionAgent(
    kernel=kernel,
    service_id="openai",
    name="Pharmacien",
    instructions=PHARMACIST_PROMPT,
)


## Définition d'une stratégie de terminaison

In [26]:
class MedicalTerminationStrategy(TerminationStrategy):
    async def should_terminate(self, agent, history):
        return len(history) >= 6  # On limite à 6 échanges


## Création du chat groupé avec stratégie de terminaison

In [27]:
chat = AgentGroupChat(
    agents=[doctor_agent, ai_medical_agent, pharmacist_agent],
    termination_strategy=MedicalTerminationStrategy()  # Ajout de la stratégie
)

## Fonction pour exécuter le dialogue

In [28]:
async def run_medical_chat():
    logger.info("🚀 Début de la consultation médicale IA")
    chat_history = ChatHistory()
    
    symptoms = input("Décrivez vos symptômes : ")
    chat_history.add_user_message(symptoms)
    
    while True:
        async for message in chat.invoke():
            logger.info(f"[{message.role}] {message.name}: {message.content}")
            print(f"{message.name}: {message.content}")
            
            if message.name not in ["Pharmacien"]:
                user_response = input("👉 Votre réponse : ")
                chat_history.add_user_message(user_response)
        
        if chat.is_complete:
            break
    
    logger.info("🏥 Consultation terminée.")


In [ ]:
await run_medical_chat()

2025-02-14 09:38:10,977 [INFO] 🚀 Début de la consultation médicale IA
